# Project 5. Utilize sensors to execute perception tasks and their applications in intelligent systems
## Project 5.1
To provide a comprehensive real-life Python code example for utilizing sensors to execute perception tasks, I'll demonstrate a simple implementation using a camera sensor and OpenCV library for object detection. This code will detect objects in a live video stream from the camera.

In [ ]:
import cv2

# Load pre-trained model and configuration files
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Set up camera capture
cap = cv2.VideoCapture(0)

while True:
    # Read frames from the camera
    ret, frame = cap.read()

    # Perform object detection
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward()

    # Process the detected objects
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Apply non-maximum suppression to remove redundant overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels on the frame
    for i in indices:
        i = i[0]
        box = boxes[i]
        left, top, width, height = box
        label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)
        cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Object Detection', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


In this code snippet, we utilize the camera sensor to capture a live video stream. The OpenCV library is used to perform object detection using the YOLOv3 model. The pre-trained model files (.cfg and .weights) need to be downloaded and provided. The coco.names file contains the names of the classes that the model can detect.

The code sets up the camera capture and continuously reads frames from the camera. It then performs object detection on each frame using the YOLOv3 model. The detected objects are processed, and non-maximum suppression is applied to remove redundant bounding boxes. Finally, the code draws the bounding boxes and labels on the frame and displays it.

## Project 5.2
Here's another comprehensive real-life Python code example for utilizing sensors to execute perception tasks. This time, we'll focus on using a Lidar sensor for obstacle detection and mapping in a simulated robotic environment using the ROS (Robot Operating System) framework.

In [ ]:
import rospy
from sensor_msgs.msg import LaserScan

def lidar_callback(msg):
    # Process Lidar data
    ranges = msg.ranges
    intensities = msg.intensities

    # Perform obstacle detection and mapping
    obstacles = []
    for i in range(len(ranges)):
        if ranges[i] < 2.0 and intensities[i] > 0.5:
            obstacle = (ranges[i], i)
            obstacles.append(obstacle)

    # Print detected obstacles
    for obstacle in obstacles:
        distance, angle = obstacle
        print(f"Detected obstacle at distance: {distance:.2f} meters, angle: {angle} degrees")

def main():
    # Initialize ROS node and subscribe to Lidar topic
    rospy.init_node('lidar_obstacle_detection')
    rospy.Subscriber('/laser_scan_topic', LaserScan, lidar_callback)

    # Spin the node to receive messages
    rospy.spin()

if __name__ == '__main__':
    try:
        main()
    except rospy.ROSInterruptException:
        pass


In this code snippet, we use the ROS framework to interact with a Lidar sensor in a simulated robotic environment. The code subscribes to the /laser_scan_topic topic, which publishes the LaserScan messages containing the data from the Lidar sensor.

The lidar_callback function is called whenever a new Lidar message is received. Inside the callback, we extract the ranges and intensities data from the message. We then process this data to perform obstacle detection and mapping. In this example, we consider objects as obstacles if they are within 2.0 meters and have an intensity greater than 0.5.

The detected obstacles are stored in a list and printed to the console. However, in a real-world scenario, you might perform additional actions like obstacle avoidance, mapping, or path planning based on the detected obstacles.

The main function initializes the ROS node, subscribes to the Lidar topic, and spins the node to receive messages continuously.

Please note that this example assumes you have a ROS environment set up, including a simulated robotic environment with a Lidar sensor publishing data on the specified topic. Additionally, you may need to configure the code to match your specific setup, such as adjusting the topic name or obstacle detection criteria, depending on the specifications of your Lidar sensor and the requirements of your application.

We assume that you have already set up a simulated robotic environment with the ROS framework. To run this code, you will need to make the following adjustments:

1 - Install ROS: Follow the instructions specific to your operating system from the official ROS website (http://www.ros.org/install).

2 - Set Up a ROS Workspace: Create a new workspace or use an existing one using the following commands in the terminal:

> $ mkdir -p ~/catkin_ws/src

> $ cd ~/catkin_ws/

> $ catkin_make

3 - Create a Package: Inside the src directory of your workspace, create a new package for your project using the following command:



> $ cd ~/catkin_ws/src

> $ catkin_create_pkg lidar_obstacle_detection rospy sensor_msgs


4 - Download Lidar Data: Download and extract the sample lidar data for testing from the following link: Sample Lidar Data.

5 - Modify the Code: Replace the /laser_scan_topic with the appropriate topic name for your Lidar sensor. Update the logic for obstacle detection and mapping according to your specific requirements.

6 - Build and Run the Code: Build your workspace and run the code using the following commands: